# #imports

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import psycopg2
from datetime import datetime, timedelta

In [2]:
conn_params = {
    "host" : "srvdados",
    "database" : "postgres",
    "user":"compras",
    "password": "pecist@compr@s2024"
}

In [3]:
query = """
    select 
        codpro, codfor, 
        num_fab
    from"D-1".produto
"""

In [ ]:
query2 = """
SELECT 
    pp.cod_pro,
    pp.dt_emissao,
    pp.qtde_ven AS vendas
FROM "D-1".prod_ped pp
JOIN "D-1".cliente c 
    ON pp.codcli = c.codcli
WHERE 
    pp.tipped = 'V' 
    AND pp.dt_emissao >= CURRENT_DATE - INTERVAL '2 years'
    AND c.codcli NOT IN ('13996','16100','18400','20000','23000','02608','24000','00270','20690','20691','20692','20693','23011')
    AND pp.codvde NOT IN ('0100','0001','0006','2319')
    AND c.codarea <> '112'
    AND c.codcid <> '0501'
    AND pp.cd_loja != '08';
"""

In [5]:
query3 = """
    select 
        codpro,  
        sum(estoque) as estoque
    from "D-1".prd_loja
    where cd_loja != '08'
    group by codpro
"""

In [6]:
query4 = """
select 
     cd_produto, min(dt_estoque) as min_dt_estoque
	 from "D-1".estoque
     where cd_loja != '08'
     group by cd_produto
"""

In [7]:
with psycopg2.connect(**conn_params) as conn:
    produtos = pd.read_sql(query, conn)

C:\Users\priscila.marques\AppData\Local\Temp\ipykernel_12668\3954374250.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  produtos = pd.read_sql(query, conn)


In [8]:
with psycopg2.connect(**conn_params) as conn:
    vendas = pd.read_sql(query2, conn)

C:\Users\priscila.marques\AppData\Local\Temp\ipykernel_12668\1012876541.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vendas = pd.read_sql(query2, conn)


In [9]:
with psycopg2.connect(**conn_params) as conn:
    estoques = pd.read_sql(query3, conn)

C:\Users\priscila.marques\AppData\Local\Temp\ipykernel_12668\538294193.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  estoques = pd.read_sql(query3, conn)


In [10]:
with psycopg2.connect(**conn_params) as conn:
    dt_estoque = pd.read_sql(query4, conn)

C:\Users\priscila.marques\AppData\Local\Temp\ipykernel_12668\2495897565.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dt_estoque = pd.read_sql(query4, conn)


# #indicadores 6 meses

In [11]:
um_ano_atras = datetime.today() - timedelta(days=365)
vendas_ultimo_ano = vendas[vendas['dt_emissao'] >= um_ano_atras]  #vendas dos último ano

In [12]:
produtos_vendidos = vendas_ultimo_ano['cod_pro'].unique()

In [13]:
produtos_sem_venda = produtos[~produtos['codpro'].isin(produtos_vendidos)] #identificar produtos sem vendas

In [14]:
produtos_com_estoque = pd.merge(produtos_sem_venda, estoques, left_on='codpro', right_on='codpro', how='left')
produtos_com_estoque = produtos_com_estoque[produtos_com_estoque['estoque'] > 0]  

In [15]:
produtos_com_estoque = pd.merge(produtos_com_estoque, dt_estoque, left_on='codpro', right_on='cd_produto', how='left')

In [16]:
produtos_antigos = produtos_com_estoque[produtos_com_estoque['min_dt_estoque'] <= um_ano_atras]

In [17]:
vendas["dt_emissao"] = pd.to_datetime(vendas["dt_emissao"])

In [19]:
quantidade_produtos = produtos_antigos.shape[0]
lista_produtos = produtos_antigos[['codpro', 'codfor', 'estoque', 'min_dt_estoque']]

print("Quantidade de produtos nessa situação:", quantidade_produtos)
print(lista_produtos)

Quantidade de produtos nessa situação: 4661
      codpro codfor  estoque min_dt_estoque
0     000093  00385     21.0     2013-05-22
1     000153  00265      2.0     2013-04-19
2     000279  00124      1.0     2013-03-07
3     000368  00265      1.0     2013-03-12
4     000432  00150      7.0     2013-08-14
...      ...    ...      ...            ...
5592  090432  00493     10.0     2024-08-27
5593  090438  00493     10.0     2024-08-27
5601  090580  00451      1.0     2024-09-10
5664  092428  01148      1.0     2024-09-05
5665  092430  01148      1.0     2024-09-05

[4661 rows x 4 columns]


In [20]:
seismeses_atras = datetime.today() - timedelta(days=180)
vendas_6meses = vendas[vendas['dt_emissao'] >= seismeses_atras]

In [21]:
produtos_vendidos6 = vendas_6meses['cod_pro'].unique()

In [22]:
produtos_sem_venda6 = produtos[~produtos['codpro'].isin(produtos_vendidos6)]

In [23]:
#Mantém somente os que têm estoque > 0.
produtos_com_estoque6 = pd.merge(produtos_sem_venda6, estoques, left_on='codpro', right_on='codpro', how='left')
produtos_com_estoque6 = produtos_com_estoque6[produtos_com_estoque6['estoque'] > 0]

In [24]:
produtos_com_estoque6 = pd.merge(produtos_com_estoque6, dt_estoque, left_on='codpro', right_on='cd_produto', how='left')

In [25]:
#Mantém os que têm estoque registrado há ≥ 6 meses (min é a data mínima de estoque).
produtos_antigos6 = produtos_com_estoque6[produtos_com_estoque6['min_dt_estoque'] <= seismeses_atras]

In [26]:
quantidade_produtos6 = produtos_antigos6.shape[0]
lista_produtos6 = produtos_antigos6[['codpro', 'codfor', 'estoque', 'min_dt_estoque']]

print("Quantidade de produtos nessa situação:", quantidade_produtos6)
print(lista_produtos6)

Quantidade de produtos nessa situação: 7787
      codpro codfor  estoque min_dt_estoque
0     000093  00385     21.0     2013-05-22
1     000153  00265      2.0     2013-04-19
2     000279  00124      1.0     2013-03-07
3     000368  00265      1.0     2013-03-12
4     000432  00150      7.0     2013-08-14
...      ...    ...      ...            ...
8586  095676  00419      6.0     2025-02-26
8605  096016  00419      4.0     2025-02-26
8608  096076  04794      1.0     2025-02-15
8614  096140  04342      5.0     2025-02-05
8673  098172  00076      6.0     2025-03-10

[7787 rows x 4 columns]


In [27]:
lista_produtos.to_csv("produtos1ano.csv")
lista_produtos6.to_csv("produtos6meses.csv")